In [1]:
import pandas as pd
import requests
import io
import os

Day = 7

# get file from website using private session key stored in enviromental variables
r = requests.get(
            f'https://adventofcode.com/2023/day/'+str(Day)+'/input',
            cookies={'session': os.getenv('AdventSessionKey')}
)

# read r.text
df = pd.read_table(io.StringIO(r.text), sep=' ', names=['hand', 'bid'])



In [2]:
# Test data
#test = "32T3K 765\nT55J5 684\nKK677 28\nKTJJT 220\nQQQJA 483\nQJJQ2 1\nJKKK2 2\nQQQQ2 1\n"
#df = pd.read_table(io.StringIO(test), sep=' ', names=['hand', 'bid'])


In [3]:
# Functions

# Define the hand_type and return the rank
# added logic for when a wildcard is provided
def hand_type(hand, wildcard=''):

    # Check if all the cards in the hand are wildcards
    if all(card == wildcard for card in hand):
        return 7  # Five of a kind

    counts = {card: hand.count(card) for card in hand}
    wildcard_count = 0  # Initialize the wildcard count

    # Check if there are wildcards in the hand
    if wildcard in counts:
        wildcard_count = counts[wildcard]  # Count the number of wildcards
        del counts[wildcard]  # Remove the count for wildcards

    # Check if there's only one unique card in the hand besides the wildcard
    if len(counts) == 1:
        card = list(counts.keys())[0]
        counts[card] += wildcard_count
        wildcard_count = 0

    # Distribute the wildcards to the other cards to maximize the hand type
    for card in sorted(counts, key=counts.get, reverse=True):
        while wildcard_count > 0 and counts[card] < 4:
            counts[card] += 1
            wildcard_count -= 1

    counts = sorted(list(counts.values()), reverse=True)

    if counts == [5]:
        return 7  # Five of a kind
    elif counts == [4, 1]:
        return 6  # Four of a kind
    elif counts == [3, 2]:
        return 5  # Full house
    elif counts == [3, 1, 1]:
        return 4  # Three of a kind
    elif counts == [2, 2, 1]:
        return 3  # Two pair
    elif counts == [2, 1, 1, 1]:
        return 2  # One pair
    else:
        return 1  # High card


# For secondary sort, score based on hand strength in base 13, then convert to base 10
def sort_cards(hand):
    base_13 = ''.join(card_values[card] for card in hand)
    score = int(base_13, 13)
    return score






In [4]:
# Part 1

# Define the card ranks
card_ranks = '23456789TJQKA'

# Define the base 13 symbols
base13_symbols = '0123456789abc'

# Assign numerical values to the cards in base 13
card_values = {card: base13_symbols[index] for index, card in enumerate(card_ranks)}

# Apply the functions to the DataFrame
df['Hand Type'] = df['hand'].apply(hand_type)
df['Score'] = df['hand'].apply(sort_cards)

# Sort the DataFrame by hand type and score
df = df.sort_values(by=['Hand Type', 'Score'])

# Reset the index and add 1 to get the rank
df.reset_index(drop=True, inplace=True)
df.index += 1

# Calculate the sum of rank * bid for each row
total = (df.index * df['bid']).sum()

print(f'The sum of rank * bid for each row is: {total}')


The sum of rank * bid for each row is: 250232501


In [5]:
# Part 2. add optional wildcard arguement, set it to 'J'

# Redefine the card ranks
card_ranks = 'J23456789TQKA'

# Define the base 13 symbols
base13_symbols = '0123456789abc'

# Assign numerical values to the cards in base 13
card_values = {card: base13_symbols[index] for index, card in enumerate(card_ranks)}

# Apply the functions to the DataFrame
df['Hand Type'] = df['hand'].apply(lambda hand: hand_type(hand, wildcard='J'))
df['Score'] = df['hand'].apply(sort_cards)

# Sort the DataFrame by hand type and score
df = df.sort_values(by=['Hand Type', 'Score'])

# Reset the index and add 1 to get the rank
df.reset_index(drop=True, inplace=True)
df.index += 1

# Calculate the sum of rank * bid for each row
total = (df.index * df['bid']).sum()

print(f'The sum of rank * bid for each row is: {total}')

# not 249141853

The sum of rank * bid for each row is: 249138943
